In [1]:
import ujson as js
import mysql.connector as mdb

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

In [2]:
def mat_eff_calc(mat_quant, me_level):
    quant_actual = np.maximum(1, np.ceil(np.round(mat_quant * ((100 - me_level) / 100), 2))).astype(int)
    return quant_actual

In [3]:
mat_region_ids = 10000043
sale_region_ids = 10000052
h = 14
market_start = '2019-01-01'

In [4]:
class Market:
    def __init__(self, mat_region_ids, sale_region_ids, horizon,
                 market_start='2015-04-01',
                 sql_script_path='./utils/sql_calls.json',
                 maria_login_path='./../settings/maria_login.json'
                ):
        if type(mat_region_ids) in (str, int): mat_region_ids = [mat_region_ids]
        if type(sale_region_ids) in (str, int): sale_region_ids = [sale_region_ids]
        
        self.mat_region_ids = mat_region_ids
        self.sale_region_ids = sale_region_ids
        self.h = horizon
        self.market_start = market_start
        
        self.conn = {}
        self.connect_maria(maria_login_path)
        
        with open(sql_script_path) as file:
            self.sql = js.load(file)
        
    def connect_maria(self, maria_login_path):
        with open(maria_login_path) as file:
            self.conn['maria'] = mdb.connect(**js.load(file))

    def get_corp_bps(self):
        corp_bps = pd.read_sql(sql['corp']['bpos'], conn, index_col='bp_item_id')
        corp_bps = corp_bps.loc[corp_bps[['bp_type_id', 'mat_eff', 'time_eff']].drop_duplicates().index]
        corp_bps.index = corp_bps.index.astype(str)
        return corp_bps

In [5]:
market = Market(mat_region_ids, sale_region_ids, h, market_start=market_start)

In [6]:
sql = {
    'offices': """\
        SELECT CorpAssets.location_id AS structure_id
        FROM Corp_Assets AS CorpAssets
        WHERE CorpAssets.type_id = 27
    ;""",
    'structures': """\
        SELECT Struct.structure_item_id AS structure_id,
            Struct.structure_name AS struct_name,
            Struct.type_id,
            Types.type_name,
            Struct.system_id,
            Sys.system_name,
            0 AS owned_station
        FROM PublicStructures AS Struct
        JOIN Types
            ON Struct.type_id = Types.type_id
        JOIN MapSystems AS Sys
            ON Struct.system_id = Sys.system_id
        WHERE Struct.structure_item_id IN ({struct_item_ids})

        UNION

        SELECT Assets.item_id AS structure_id,
            Assets.item_name as struct_name,
            Assets.type_id,
            Types.type_name,
            Assets.location_id AS system_id,
            Sys.system_name,
            1 AS owned_station
        FROM Corp_Assets AS Assets
        JOIN Types
            ON Assets.type_id = Types.type_id
        JOIN MapSystems AS Sys
            ON Assets.location_id = Sys.system_id
        WHERE Assets.item_id IN ({struct_item_ids})
    ;"""
}

In [7]:
corp_offices = pd.read_sql(sql['offices'], market.conn['maria'])

corp_offices = pd.read_sql(
    sql['structures'].format(
        struct_item_ids=','.join(corp_offices['structure_id'].astype(str))
    ),
    market.conn['maria'],
    index_col='structure_id'
)

corp_offices

,struct_name,type_id,type_name,system_id,system_name,owned_station
structure_id,,,,,,
1030653708061,Chanoun - 2% Tax - Gloria's Bar And Grill,35827,Sotiyo,30004122,Chanoun,0
1031240032544,Jakri - Tugboat,35833,Fortizar,30004125,Jakri,0
1027327111023,Jakri - The Refining Forge,35835,Athanor,30004125,Jakri,1
1031061348700,Jakri - The Industrial Forge,35825,Raitaru,30004125,Jakri,1
